## Building a SQL database for PurpleAir 

#### Author: Yunha Lee, Ph.D. 
#### Date  : Dec 01, 2022

The code below is designed to create a database using SQLite in your local computer. It takes real-time data from a purple air sensor and stores in the database.  PurpleAir limits public access to their historical database, so this allows to build your own sensor database. 

* The current version uses the "purpleair" library from GitHub, which handles Purple Air API: https://github.com/csm10495/purpleair
* SQLite is chosen for this because 

##### Side Note about PurpleAir database

PurpleAir has recently changed the database access methods.  

As of December, 2022, the recommended methods to get PurpleAir data are the below:

1) using their "Sensor data download tool" website (without API)
    see link: https://www.purpleair.com/sensorlist
2) using real-time API to create your own "personal" database, instead of using their historical API
    see link: https://community.purpleair.com/t/historical-api-endpoints-are-now-restricted/1557
3) using the historical API occasionally to get the missed data
    email contact@purpleair.com to get a permission to use the history API. 


*The jupyter notebook is designed to support the method "2", building your own database in a local computer.*

In [95]:
from purpleair import PurpleAir
import requests
import pandas as pd
import os
from datetime import datetime
import time
import json
from io import StringIO
import sqlalchemy as db
import schedule

def flatten_json(data: dict) -> dict:
  
    flat_data = dict()
    for key, value in data.items():
        if not isinstance(value, dict):
            flat_data[key] = value
        else:

            # 1st nested item
            for k, v in value.items():
                if (k == "stats_a"):
                    print("not saving stats_a information ", k)
                else:

                    if not isinstance(v, dict):
                        flat_data[key + "_" + k] = v
                    else:

                        # 2nd nested items
                        for k2, v2 in v.items():
                            if not isinstance(v2, dict):
                                flat_data[key + "_"+ k+ "_"+ k2] = v2
                            else: 

                                # 3rd nested items
                                for k3, v3 in v.items():
                                    if not isinstance(v3, dict):
                                        flat_data[key + "_"+ k+ "_"+ k2+ "_"+ k3] = v3
                                    else:
                                        print("WARNING: this json file has more than 3rd nested layers")
      
    return flat_data

def get_PA_data(key_read, sensor_id):

    """
    Parameters
    ----------
    key_read : your PurpleAir API key 
    sensor_id : ID number of the sensor you want to read
    """

    # get real-time sensor data 
    p = PurpleAir(key_read)
    json_data = p.get_sensor_data(sensor_id)

    # Flatten multi-nested json data to a dictionary
    flat_data = flatten_json(data= json_data)

    # Create a pandas dataframe 
    df = pd.DataFrame(flat_data, index=[0])

    return df

def create_sql_csv (save_as_sql=True, save_as_csv=True):
    """
    Parameters
    ----------
    key_read : your PurpleAir API key 
    sensor_id : ID number of the sensor you want to read
    db_path : database directory path
    engine : SQL engine info
    save_as_sql : Boolean value to save the data as SQLite 
    save_as_csv : Boolean value to save the data as CSV
    """

    df = get_PA_data(key_read, sensor_id)

    # Writing to SQLite Table (Optional)
    sql_tablename = 'purpleair_' + sensor_id
    df.to_sql(sql_tablename, con=engine, if_exists='replace', index=True)

    # writing to csv file
    csv_filename = db_path + '/sensor_index_' + sensor_id + '.csv'
    df.to_csv(csv_filename, index=False, header=True)

    return

def append_sql_csv (save_as_sql=True, save_as_csv=True):
    """
    Parameters
    ----------
    key_read : your PurpleAir API key 
    sensor_id : ID number of the sensor you want to read
    db_path : database directory path
    engine : SQL engine info
    save_as_sql : Boolean value to save the data as SQLite 
    save_as_csv : Boolean value to save the data as CSV
    """

    df = get_PA_data(key_read, sensor_id)
    print(df)

    # Writing to SQLite Table (Optional)
    sql_tablename = 'purpleair_' + sensor_id
    df.to_sql(sql_tablename, con=engine, if_exists='append', index=False)

    # writing to csv file
    csv_filename = db_path + '/sensor_index_' + sensor_id + '.csv'
    df.to_csv(csv_filename, index=False, header=False, mode='a')

    return

In [96]:
# set a sensor index 
sensor_id = '143856'

# minutes time frequency of generating PA real-time data
pa_time = 2 # according to PA, every 2 minutes

# API Keys provided by PurpleAir(c)
key_dir = "./purpleair_api_read_key"
with open(key_dir) as f:
    key_read = f.readlines()[0]
    key_read = key_read.strip() # could contain whitespace
    print(key_read, len(key_read))


# create database directory
db_path = "./database_" + datetime.now().strftime("%b-%d-%Y-%H_%M_%S")
#db_path = "./database"  # _" + datetime.now().strftime("%d-%m-%Y-%H_%M_%S")


print("database dir is created : ", db_path)
if not os.path.exists(db_path):
    os.makedirs(db_path)

# Starting engine for SQLite
sql_path = db_path + "/purpleair_"+ sensor_id +".sqlite"
print(f'os.path.abspath(sql_path): {str(os.path.abspath(sql_path))}')
SQL_URI = "sqlite:///" + os.path.abspath(sql_path) 
engine = db.create_engine(SQL_URI)

# create sql and/or csv files
create_sql_csv(save_as_sql=True, save_as_csv=True)

# calling append_sql_csv every two minutes to create historical database from real-time data
schedule.every(pa_time).minutes.do(append_sql_csv, save_as_sql=True, save_as_csv=True)

#while True:
#    schedule.run_pending()
#    time.sleep(60)

F2324129-7000-11ED-B6F4-42010A800007 36
database dir is created :  ./database_Mar-16-2023-16_45_50
os.path.abspath(sql_path): /Users/yunhalee/Desktop/DLair/purpleair_indoor_aqi/database_Mar-16-2023-16_45_50/purpleair_143856.sqlite
not saving stats_a information  stats_a


Every 2 minutes do append_sql_csv(save_as_sql=True, save_as_csv=True) (last run: [never], next run: 2023-03-16 16:47:50)

In [99]:
# read all sql database for testing  
test = pd.read_sql('purpleair_'+sensor_id, engine)
test.head()

[(0, 'V1.0.11-0.0.42', 1678999550, 1678999526, 143856, 1657586037, 1643666340, 1678999510, 0, 0, 'Bren Mar Park', 0, 1, 'PA-I', '2.0+BME280+PMSX003-A', 35, '7.02', -55, 2648, 191, 16192, 5, 38.797955, -77.15579, 198, 1, 0, 0, 0, 30, 12, 12, 79, 79, 1010.13, 1010.13, 0.02, 0.0, 0.0, 0.0, 0.0, 0.6, 0.6, 0.0, 0.0, 2.0, 2.0, 2.2, 2.2, 313.3, 313.3, 136, 136, 37, 37, 5, 5, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1644241, 'SKCQN9XL3HRMIO03', 1644243, 'WBOFV6KCLBHSDLEK', 1644242, 'IZB5PHB5U8VGOUI2', 1644244, '7LNT1JWRO1DH2ME1', 0.0, 0.1, 0.4, 0.6, 0.5, 1.2, 2.9, 1678999510)]

In [100]:
# read CSV files
csv_filename = db_path + '/sensor_index_' + sensor_id + '.csv'
df = pd.read_csv(csv_filename)
print(df)

      api_version  time_stamp  data_time_stamp  sensor_sensor_index  \
0  V1.0.11-0.0.42  1678999550       1678999526               143856   

   sensor_last_modified  sensor_date_created  sensor_last_seen  \
0            1657586037           1643666340        1678999510   

   sensor_private  sensor_is_owner    sensor_name  ...  sensor_secondary_id_b  \
0               0                0  Bren Mar Park  ...                1644244   

   sensor_secondary_key_b sensor_stats_pm2.5 sensor_stats_pm2.5_10minute  \
0        7LNT1JWRO1DH2ME1                0.0                         0.1   

   sensor_stats_pm2.5_30minute  sensor_stats_pm2.5_60minute  \
0                          0.4                          0.6   

   sensor_stats_pm2.5_6hour  sensor_stats_pm2.5_24hour  \
0                       0.5                        1.2   

   sensor_stats_pm2.5_1week  sensor_stats_time_stamp  
0                       2.9               1678999510  

[1 rows x 90 columns]


In [102]:
# remove the database directory 

for f in os.listdir(db_path):
    os.remove(os.path.join(db_path, f))

os.rmdir(db_path)